# Bert Centralised Learning

Set the module directory to import python files (RUN JUST ONCE)

In [1]:

import os
os.chdir('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your project
import sys
sys.path.append('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your models directory
print(sys.path)

%load_ext autoreload
%autoreload 2



['/home/victor/_bcfl/fabric-federated-learning/federated-learning/client/notebooks', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python311.zip', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/lib-dynload', '', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/site-packages', '/home/victor/_bcfl/fabric-federated-learning/federated-learning']


In [ ]:
# Running in colaboratory
!git clone https://github.com/vdevictor96/fabric-federated-learning
%cd fabric-federated-learning/federated-learning/
!git pull


In [2]:
import torch
print(f"Cuda available: {torch.cuda.is_available()}")
# Get the name of the CUDA device
print(torch.cuda.get_device_name(0))

try:
    print(
        f"major and minor cuda capability of the device: {torch.cuda.get_device_capability()}")
except Exception:
    print("No Cuda available")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Check if CUDA is available and set the default tensor type to CUDA
# print('Using device: %s' % device)
# if torch.cuda.is_available():
#     torch.set_default_device('cuda')
#     print("Cuda set as default device")
# else:
#     torch.set_default_device('cpu')
#     print("Cuda not available, CPU set as default device")

Cuda available: True
NVIDIA GeForce MX150
major and minor cuda capability of the device: (6, 1)


In [46]:
# NVIDIA drivers not working
torch.set_default_device('cpu')
device = torch.device('cpu')

### Initialize variables

In [3]:
from client.model.bert_tiny import get_bert_tiny_tokenizer

MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
EVAL_BATCH_SIZE = 2
TEST_BATCH_SIZE = 8
NUM_EPOCHS = 10
LEARNING_RATE = 6e-05
tokenizer = get_bert_tiny_tokenizer()
root = 'client/data/datasets/dep1_cleaned.csv'



/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading Reddit Depression Dataloaders

In [5]:
from client.data.reddit_dep import get_reddit_dep_dataloaders


train_loader, eval_loader, test_loader = get_reddit_dep_dataloaders(root, tokenizer, 
                                                                    train_size=0.2,
                                                                    eval_size=0.2,
                                                                    test_size=0.6,
                                                                    train_batch_size=TRAIN_BATCH_SIZE, 
                                                                    eval_batch_size=EVAL_BATCH_SIZE, 
                                                                    test_batch_size=TEST_BATCH_SIZE, 
                                                                    max_len=MAX_LEN,
                                                                    seed=200, 
                                                                    device=device)


### Creating Bert Tiny Model

In [6]:
from client.model.bert_tiny import get_bert_tiny_model
from client.utils import set_seed

set_seed(200)

bert_tiny = get_bert_tiny_model(device=device)
print(bert_tiny)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, e

### Train model

#### Define optimizer

In [7]:
from torch.optim import AdamW
from transformers import get_scheduler



# Optimizer and learning rate scheduler
optimizer = AdamW(bert_tiny.parameters(), lr=LEARNING_RATE)

# Scheduler
num_training_steps = NUM_EPOCHS * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)


#### Training loop

In [10]:

from client.train import train_text_class
modelpath = 'client/models/bert-tiny'
modelname = 'bert_tiny'
train_text_class(bert_tiny, modelpath, modelname, train_loader, eval_loader, optimizer, LEARNING_RATE, lr_scheduler, NUM_EPOCHS, device, eval_flag=True, progress_bar_flag=False)


Epoch [1/10], Step [100/155], Loss: 0.1488, Accuracy: 96.25 %
-------------------------------
Epoch [1/10], Loss: 0.1615, Accuracy: 95.15 %
-------------------------------
Validation Loss: 0.5045, Validation Accuracy: 78.84 %
-------------------------------
Epoch [2/10], Step [100/155], Loss: 0.1780, Accuracy: 93.75 %
-------------------------------
Epoch [2/10], Loss: 0.1607, Accuracy: 94.67 %
-------------------------------
Validation Loss: 0.5045, Validation Accuracy: 78.84 %
-------------------------------
Epoch [3/10], Step [100/155], Loss: 0.1667, Accuracy: 95.25 %
-------------------------------
Epoch [3/10], Loss: 0.1601, Accuracy: 95.80 %
-------------------------------
Validation Loss: 0.5046, Validation Accuracy: 78.84 %
-------------------------------
Epoch [4/10], Step [100/155], Loss: 0.1715, Accuracy: 95.25 %
-------------------------------
Epoch [4/10], Loss: 0.1651, Accuracy: 95.48 %
-------------------------------
Validation Loss: 0.5046, Validation Accuracy: 78.84 %


OrderedDict([('bert.embeddings.word_embeddings.weight',
              tensor([[-4.0998e-03, -3.0680e-02, -3.5279e-03,  ...,  1.8916e-02,
                        3.7378e-03, -2.9220e-03],
                      [-4.2728e-04, -3.6911e-02, -1.7160e-02,  ...,  2.9300e-02,
                       -1.0393e-02,  2.6759e-02],
                      [ 5.9389e-03,  4.2099e-03, -1.9557e-02,  ...,  1.6791e-02,
                       -2.7789e-02, -6.8984e-03],
                      ...,
                      [ 3.5556e-02, -1.5883e-02,  4.9928e-03,  ...,  5.4046e-03,
                       -1.1265e-02, -6.9495e-05],
                      [-8.6977e-03, -2.2506e-02,  3.1978e-03,  ...,  2.7578e-02,
                       -1.9545e-02,  2.4011e-03],
                      [-7.8867e-02, -7.5372e-02, -4.6638e-03,  ..., -5.3315e-03,
                       -4.4973e-02,  5.9813e-02]], device='cuda:0')),
             ('bert.embeddings.position_embeddings.weight',
              tensor([[ 0.0439, -0.0150, -0.5180,  

### Load best model

In [ ]:
from client.model.bert_tiny import load_bert_tiny_model

best_bert_tiny = load_bert_tiny_model('client/models/bert-tiny/bert_tiny_best.ckpt', device=device)

### Test the best saved model

In [12]:
from client.test import test_text_class

# Test the model
test_text_class(best_bert_tiny, test_loader, device=device)

Accuracy of the network on the 1858 test sentences: 77.07 %


77.07212055974165